In [1]:
!pip install torchinfo

In [2]:
import timm
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split, ConcatDataset
import numpy as np
from tqdm import tqdm
import torch.nn.functional as F
import itertools
from torchinfo import summary

In [9]:
model = timm.create_model('swin_base_patch4_window7_224', pretrained=False, num_classes=37)
print(summary(model, input_size=(32, 3, 224, 224)))

Layer (type:depth-idx)                             Output Shape              Param #
SwinTransformer                                    [32, 37]                  --
├─PatchEmbed: 1-1                                  [32, 56, 56, 128]         --
│    └─Conv2d: 2-1                                 [32, 128, 56, 56]         6,272
│    └─LayerNorm: 2-2                              [32, 56, 56, 128]         256
├─Sequential: 1-2                                  [32, 7, 7, 1024]          --
│    └─SwinTransformerStage: 2-3                   [32, 56, 56, 128]         --
│    │    └─Identity: 3-1                          [32, 56, 56, 128]         --
│    │    └─Sequential: 3-2                        [32, 56, 56, 128]         397,896
│    └─SwinTransformerStage: 2-4                   [32, 28, 28, 256]         --
│    │    └─PatchMerging: 3-3                      [32, 28, 28, 256]         132,096
│    │    └─Sequential: 3-4                        [32, 28, 28, 256]         1,582,224
│    └─SwinTra

In [10]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

batch_size = 32
learning_rate = 0.001
num_epochs = 50

In [19]:
trainval_data = datasets.OxfordIIITPet(root="data", split="trainval", target_types="category", download=True, transform=transform)
test_data = datasets.OxfordIIITPet(root="data", split="test", target_types="category", download=True, transform=transform)
combined_data = ConcatDataset([trainval_data, test_data])

train_size = int(0.7 * len(combined_data))
val_size = int(0.15 * len(combined_data))
test_size = len(combined_data) - train_size - val_size
train_data, val_data, test_data = random_split(combined_data, [train_size, val_size, test_size])

In [20]:
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

print(f"Train set size: {len(train_data)}")
print(f"Validation set size: {len(val_data)}")
print(f"Test set size: {len(test_data)}")

Train set size: 5144
Validation set size: 1102
Test set size: 1103


In [21]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [22]:
def train(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in tqdm(train_loader, desc="Training"):
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    accuracy = 100 * correct / total
    print(f"Train Loss: {epoch_loss:.4f}, Train Accuracy: {accuracy:.2f}%")

In [23]:
def evaluate(model, data_loader, criterion, device, phase="Validation"):
    model.eval()
    running_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in tqdm(data_loader, desc=f"{phase}"):
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)

            running_loss += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(data_loader)
    accuracy = 100 * correct / total
    print(f"{phase} Loss: {epoch_loss:.4f}, {phase} Accuracy: {accuracy:.2f}%")

In [24]:
def measure_inference_time(model, data_loader, device):
    model.eval()
    times = []

    with torch.no_grad():
        for inputs, _ in data_loader:
            inputs = inputs.to(device)
            start_time = torch.cuda.Event(enable_timing=True)
            end_time = torch.cuda.Event(enable_timing=True)

            start_time.record()
            _ = model(inputs)  # inference 수행
            end_time.record()

            # 시간 측정
            torch.cuda.synchronize()  # CUDA에서 모든 커널이 완료될 때까지 대기
            elapsed_time = start_time.elapsed_time(end_time)  # 밀리초 단위로 반환
            times.append(elapsed_time)

    # 통계량 계산
    times_np = np.array(times)
    total_inferences = len(times_np)
    avg_time = np.mean(times_np)
    std_dev = np.std(times_np)
    max_time = np.max(times_np)
    min_time = np.min(times_np)

    # 결과 출력
    print(f"Inference Time Measurement Results:")
    print(f"Total Inferences: {total_inferences}")
    print(f"Average Time: {avg_time:.2f} ms")
    print(f"Standard Deviation: {std_dev:.2f} ms")
    print(f"Maximum Time: {max_time:.2f} ms")
    print(f"Minimum Time: {min_time:.2f} ms")

    return times

In [25]:
for epoch in range(num_epochs):
    print(f"\nEpoch {epoch+1}/{num_epochs}")
    train(model, train_loader, criterion, optimizer, device)
    evaluate(model, val_loader, criterion, device, phase="Validation")


Epoch 1/50


Training: 100%|██████████| 161/161 [01:06<00:00,  2.43it/s]


Train Loss: 3.6635, Train Accuracy: 3.07%


Validation: 100%|██████████| 35/35 [00:08<00:00,  4.33it/s]


Validation Loss: 3.6220, Validation Accuracy: 3.18%

Epoch 2/50


...

Validation Loss: 3.6183, Validation Accuracy: 2.54%

Epoch 50/50


Training: 100%|██████████| 161/161 [01:06<00:00,  2.43it/s]


Train Loss: 3.6124, Train Accuracy: 2.66%


Validation: 100%|██████████| 35/35 [00:07<00:00,  4.38it/s]

Validation Loss: 3.6175, Validation Accuracy: 2.54%


In [26]:
print("\nFinal Test Evaluation")
evaluate(model, test_loader, criterion, device, phase="Test")


Final Test Evaluation


Test: 100%|██████████| 35/35 [00:08<00:00,  4.29it/s]

Test Loss: 3.6203, Test Accuracy: 1.81%


In [27]:
times = measure_inference_time(model, test_loader, device)

Inference Time Measurement Results:
Total Inferences: 35
Average Time: 78.36 ms
Standard Deviation: 6.76 ms
Maximum Time: 84.97 ms
Minimum Time: 40.41 ms
